In [1]:
import numpy as np
import pandas as pd

from collab2.foraging.toolkit import (
    dataObject,
    filter_by_distance,
    rescale_to_grid,
    subset_frames_evenly_spaced,
)

In [2]:
# create a test object

num_frames = 10
num_foragers = 5

np.random.seed(42)

data = {
    "x": np.random.randn(num_frames * num_foragers),
    "y": np.random.randn(num_frames * num_foragers),
    "time": np.tile(np.arange(num_frames), num_foragers),
    "forager": np.concatenate([i * np.ones(num_frames) for i in range(num_foragers)]),
}
foragersDF = pd.DataFrame(data)

# add nan values
nan_ind = [4, 9, 20]
foragersDF.loc[nan_ind, ["x", "y"]] = np.nan

# remove values for certain time points
drop_ind = [12, 6, 14]
drop_t = foragersDF["time"][drop_ind].to_list()
drop_f = foragersDF["forager"][drop_ind].to_list()
foragersDF = foragersDF.drop(drop_ind)

foragersDF_scaled = rescale_to_grid(foragersDF, size=10)
foragersDF_scaled_subsampled = subset_frames_evenly_spaced(foragersDF_scaled, 5)

foragers_object = dataObject(foragersDF_scaled_subsampled)

original_frames: 9
original_shape: (47, 4)
resulting_frames: 5
resulting_shape: (25, 4)
min_time 1
max_time 5


/Users/Admin/Repositories/collab-creatures/collab2/foraging/toolkit/utils.py:36: UserWarning: Nan values in data. Specify handling of missing data using skip_incomplete_frames argument to generate_all_predictors
  warnings.warn(f"Nan values in data. Specify handling of missing data using skip_incomplete_frames argument to generate_all_predictors")
/Users/Admin/Repositories/collab-creatures/collab2/foraging/toolkit/utils.py:44: UserWarning: Incomplete frames in data. Specify handling of missing data using skip_incomplete_frames argument to generate_all_predictors
  warnings.warn(f"Incomplete frames in data. Specify handling of missing data using skip_incomplete_frames argument to generate_all_predictors")


In [3]:
foragers_object.foragersDF

,x,y,time,forager
0,6.0,7.0,1,1.0
1,5.0,5.0,2,1.0
3,9.0,7.0,3,1.0
5,5.0,8.0,4,1.0
7,7.0,5.0,5,1.0
10,4.0,5.0,1,2.0
11,4.0,6.0,2,2.0
13,1.0,4.0,3,2.0
15,4.0,9.0,4,2.0
17,6.0,8.0,5,2.0


In [4]:
# interaction partners of forager 1 at time 1
filter_by_distance(foragers_object.foragersDF, f=1, t=1, interaction_length=3)

[2.0, 4.0, 5.0]

In [25]:
# adding constraint

# suppose we only want foragers that are on the right half of the grid

# convention : every constraint function must take a list of foragers (within interaction distance), index of focal forager, current time, foragersDF + any additional parameters


def filter_x_constraint(f_ind, f, t, foragersDF, grid_size):
    current_positions = foragersDF.loc[
        np.logical_and(foragersDF["forager"].isin(f_ind), foragersDF["time"] == t)
    ]
    f_ind_constr = current_positions.loc[
        current_positions["x"] > grid_size / 2, "forager"
    ].to_list()
    return f_ind_constr


# interaction partners of forager 1 at time 1 with constraint
filter_by_distance(
    foragers_object.foragersDF,
    f=1,
    t=1,
    interaction_length=3,
    interaction_constraint=filter_x_constraint,
    interaction_constraint_params={"grid_size": foragers_object.grid_size},
)

[5.0]